https://www.youtube.com/watch?v=eao_6ot8yH0

In [1]:
# import libraries

import tkinter as tk
from tkinter import ttk
import yfinance as yf
import mplfinance as mpf
import matplotlib.pyplot as plt
import io
from PIL import Image, ImageTk
import datetime
from tkcalendar import DateEntry
import tkinter.colorchooser as colorchooser
import ta
from tkinter import messagebox

# Global Variables

In [2]:
# Global variables to store the current state
current_symbol = None
current_start_date = None
current_end_date = None
df_full = None
addplots = []

# Functions

function for fetching and displaying charts

In [3]:
def fetch_and_display_chart(symbol, addplots=[]):
    
    global current_symbol, current_start_date, current_end_date, df_full
    
    # Define start and end dates
    start_date = start_date_entry.get_date()
    end_date = end_date_entry.get_date()
    
    # Get the desired chart style
    style = style_var.get()
    
    # Get the desired chart type
    chart_type = chart_type_var.get()
    
    # Check if data needs to be downloaded or can be reused
    if (df_full is None or symbol != current_symbol or start_date < current_start_date or end_date > current_end_date):
        
        # Determine the earliest and latest dates needed
        new_start_date = min(start_date, current_start_date) if current_start_date else start_date
        new_end_date = max(end_date, current_end_date) if current_end_date else end_date
        
        # Fetch data using yfinance for the new date range
        df_full = yf.download(symbol, start=new_start_date, end=new_end_date)
        
        # Update the current state
        current_symbol = symbol
        current_start_date = new_start_date
        current_end_date = new_end_date
        
    # Slice the existing data to match the new date range
    df = df_full.loc[start_date:end_date]
           
    # Get canvas dimensions
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    
    # Handle the different chart types
    if chart_type == 'renko' or chart_type == 'pnf':
        fig, ax = mpf.plot(df, type=chart_type,title=f'{symbol} {chart_type} Chart', style=style, returnfig=True, 
                      figsize=(canvas_width / 100, canvas_height / 100)) ## remove addplots
    else:
                    
        # Create the mplfinance figure
        fig, ax = mpf.plot(df, type=chart_type,title=f'{symbol} {chart_type} Chart', style=style, returnfig=True, 
                          figsize=(canvas_width / 100, canvas_height / 100), addplot=addplots) 
    
    # Save the figure to a bytes buffer
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    
    # Convert image bytes to a format that can be used with tkinter
    image = Image.open(buf)
    photo = ImageTk.PhotoImage(image)
    
    # Update the canvas with the new image
    canvas.create_image(0, 0, anchor=tk.NW, image=photo)
    canvas.image = photo  # Keep a reference to avoid garbage collection
    
    # Update the canvas size to fit the image
    canvas.config(scrollregion=canvas.bbox(tk.ALL))
    
    
    # Close the mplfinance figure
    plt.close(fig)

fuction for adding studies

In [4]:
def add_study():
    global study_window
    study_window = tk.Toplevel(root)
    study_window.title("Add Study")
    
    # Determine the required width and height based on the study content
    study_type = study_var.get()
    if study_type == 'Moving Average':
        add_ma_study(study_window)
    elif study_type == 'RSI':
        add_rsi_study(study_window)
    elif study_type == 'Bollinger Bands':
        add_bollinger_bands_study(study_window)
    elif study_type == 'MACD':
        add_macd_study(study_window)
    elif study_type == 'Stochastics':
        add_stochastics_study(study_window)
    
    # Resize the window to fit its contents
    study_window.update_idletasks()  # Ensure all widgets are updated
    required_width = study_window.winfo_reqwidth()
    required_height = study_window.winfo_reqheight()
    
    # Center the window on the screen
    screen_width = study_window.winfo_screenwidth()
    screen_height = study_window.winfo_screenheight()
    x = (screen_width - required_width) // 2
    y = (screen_height - required_height) // 2

    study_window.geometry(f"{required_width}x{required_height}+{x}+{y}")   

function for selecting colors

In [5]:
def select_color(color_var, parent_window, update_callback):
    color = tk.colorchooser.askcolor(parent=parent_window, title="Select Color", initialcolor=color_var.get())
    if color:
        color_var.set(color[1])
        update_callback()

function for adding the MA

In [6]:
def add_ma_study(study_window):
    ma_type_label = ttk.Label(study_window, text="Select MA Type:")
    ma_type_label.pack(pady=5)
    
    ma_type_var = tk.StringVar(value='Simple')
    ma_type_menu = ttk.Combobox(study_window, textvariable=ma_type_var, values=['Simple', 'Exponential'])
    ma_type_menu.pack(pady=5)
    
    ma_length_label = ttk.Label(study_window, text="Enter MA Length:")
    ma_length_label.pack(pady=5)
    
    ma_length_entry = ttk.Entry(study_window,justify='center')
    ma_length_entry.insert(0, "20")
    ma_length_entry.pack(pady=5)
    
    ma_color_label = ttk.Label(study_window, text="Selected MA Color:")
    ma_color_label.pack(pady=5)
    
    ma_color_var = tk.StringVar(value='black')
    selected_color_label = ttk.Label(study_window, background=ma_color_var.get(), width=2)
    selected_color_label.pack(pady=5)
    
    def update_selected_color():
        selected_color_label.config(background=ma_color_var.get()) 
        
    ma_color_button = ttk.Button(study_window, text="Choose Color", command=lambda: select_color(ma_color_var,
                                study_window, update_selected_color))
    
    ma_color_button.pack(pady=5)
    
    def on_ok():
        selected_ma_type = ma_type_var.get()
        selected_ma_length = int(ma_length_entry.get())
        selected_ma_color = ma_color_var.get()
        
        ma_column_name = f'{selected_ma_type}_MA_{selected_ma_length}'
        
        if selected_ma_type == 'Simple':
            df_full[ma_column_name] = ta.trend.sma_indicator(df_full['Close'], window=selected_ma_length)
        elif selected_ma_type == 'Exponential':
            df_full[ma_column_name] = ta.trend.ema_indicator(df_full['Close'], window=selected_ma_length)
            
        update_chart_with_study(ma_column_name, selected_ma_color, group='MA')
        study_window.destroy()
        
    
    ok_button = ttk.Button(study_window, text="OK",command=on_ok)
    ok_button.pack(pady=5) 
   

function for adding the rsi

In [7]:
def add_rsi_study(study_window):
    rsi_length_label = ttk.Label(study_window, text="Enter RSI Length:")
    rsi_length_label.pack(pady=5)

    rsi_length_entry = ttk.Entry(study_window,justify="center")
    rsi_length_entry.insert(0, "14")
    rsi_length_entry.pack(pady=5)

    rsi_color_label = ttk.Label(study_window, text="Select RSI Color:")
    rsi_color_label.pack(pady=5)

    rsi_color_var = tk.StringVar(value='black')
    selected_rsi_color_label = ttk.Label(study_window, background=rsi_color_var.get(), width=2)
    selected_rsi_color_label.pack(pady=5)
    
    def update_selected_rsi_color():
        selected_rsi_color_label.config(background=rsi_color_var.get())

    rsi_color_button = ttk.Button(study_window, text="Choose Color", command=lambda: select_color(rsi_color_var, study_window,
                                  update_selected_rsi_color))
    
    rsi_color_button.pack(pady=5)
    
    def on_ok():
        selected_rsi_length = int(rsi_length_entry.get())
        selected_rsi_color = rsi_color_var.get()
        df_full['RSI'] = ta.momentum.rsi(df_full['Close'], window=selected_rsi_length)
        update_chart_with_study('RSI', selected_rsi_color, panel=1, group='RSI')
        study_window.destroy()
        
    ok_button = ttk.Button(study_window, text="OK", command=on_ok)
    ok_button.pack(pady=10)
    
    
    

function for adding the MACD

In [8]:
def add_macd_study(study_window):
    
    macd_slow_label = ttk.Label(study_window, text="Enter MACD Slow Length:")
    macd_slow_label.pack(pady=5)

    macd_slow_entry = ttk.Entry(study_window,justify="center")
    macd_slow_entry.insert(0, "26")
    macd_slow_entry.pack(pady=5)
    
    macd_fast_label = ttk.Label(study_window, text="Enter MACD Fast Length:")
    macd_fast_label.pack(pady=5)

    macd_fast_entry = ttk.Entry(study_window,justify="center")
    macd_fast_entry.insert(0, "12")
    macd_fast_entry.pack(pady=5)
    
    macd_signal_label = ttk.Label(study_window, text="Enter MACD Signal Length:")
    macd_signal_label.pack(pady=5)

    macd_signal_entry = ttk.Entry(study_window,justify="center")
    macd_signal_entry.insert(0, "9")
    macd_signal_entry.pack(pady=5)
    
    # Label and button for selecting color for MACD line
    macd_color_label = ttk.Label(study_window, text="Select MACD Line Color:")
    macd_color_label.pack(pady=5)
    macd_color_var = tk.StringVar(value='blue')
    selected_macd_color_label = ttk.Label(study_window, background=macd_color_var.get(), width=2)
    selected_macd_color_label.pack(pady=5)
    macd_color_button = ttk.Button(study_window, text="Choose Color", command=lambda: select_color(macd_color_var,
                                   study_window, lambda: update_selected_macd_color('MACD')))
    macd_color_button.pack(pady=5)
    
    # Label and button for selecting color for MACD signal line
    macd_signal_color_label = ttk.Label(study_window, text="Select MACD Signal Line Color:")
    macd_signal_color_label.pack(pady=5)
    macd_signal_color_var = tk.StringVar(value='orange')
    selected_macd_signal_color_label = ttk.Label(study_window, background=macd_signal_color_var.get(), width=2)
    selected_macd_signal_color_label.pack(pady=5)
    macd_signal_color_button = ttk.Button(study_window, text="Choose Color", command=lambda: select_color(macd_signal_color_var,
                                          study_window, lambda: update_selected_macd_color('MACD_Signal')))
    macd_signal_color_button.pack(pady=5)
    
    # Label and button for selecting color for MACD histogram
    macd_hist_color_label = ttk.Label(study_window, text="Select MACD Histogram Color:")
    macd_hist_color_label.pack(pady=5)
    macd_hist_color_var = tk.StringVar(value='lightgray')
    selected_macd_hist_color_label = ttk.Label(study_window, background=macd_hist_color_var.get(), width=2)
    selected_macd_hist_color_label.pack(pady=5)
    macd_hist_color_button = ttk.Button(study_window, text="Choose Color", command=lambda: select_color(macd_hist_color_var,
                                        study_window, lambda: update_selected_macd_color('MACD_Hist')))
    macd_hist_color_button.pack(pady=5)
    
    def update_selected_macd_color(component):
        if component == 'MACD':
            selected_color_label = selected_macd_color_label
            selected_color_var = macd_color_var
        elif component == 'MACD_Signal':
            selected_color_label = selected_macd_signal_color_label
            selected_color_var = macd_signal_color_var
        else:
            selected_color_label = selected_macd_hist_color_label
            selected_color_var = macd_hist_color_var
            
        selected_color_label.config(background=selected_color_var.get())
        
    def on_ok():
        selected_macd_color = macd_color_var.get()
        selected_macd_signal_color = macd_signal_color_var.get()
        selected_macd_hist_color = macd_hist_color_var.get()

        selected_macd_slow = int(macd_slow_entry.get())
        selected_macd_fast = int(macd_fast_entry.get())
        selected_macd_signal = int(macd_signal_entry.get())

        macd = ta.trend.MACD(df_full['Close'], window_slow=selected_macd_slow, window_fast=selected_macd_fast,
                             window_sign=selected_macd_signal)
        df_full['MACD'] = macd.macd()
        update_chart_with_study('MACD', selected_macd_color, panel=1, group='MACD')

        df_full['MACD_Signal'] = macd.macd_signal()
        update_chart_with_study('MACD_Signal', selected_macd_signal_color, panel=1,group='MACD')

        df_full['MACD_Hist'] = macd.macd_diff()
        update_chart_with_study('MACD_Hist', selected_macd_hist_color, type='bar', width=0.8, panel=1,group='MACD')

        study_window.destroy()

    ok_button = ttk.Button(study_window, text="OK", command=on_ok)
    ok_button.pack(pady=5) 

function for adding Bollinger Bands

In [9]:
def add_bollinger_bands_study(study_window):
    bb_length_label = ttk.Label(study_window, text="Enter Bollinger Bands Length:")
    bb_length_label.pack(pady=5)

    bb_length_entry = ttk.Entry(study_window,justify="center")
    bb_length_entry.insert(0, "20")
    bb_length_entry.pack(pady=5)

    bb_stddev_label = ttk.Label(study_window, text="Enter Standard Deviation:")
    bb_stddev_label.pack(pady=5)

    bb_stddev_entry = ttk.Entry(study_window,justify="center")
    bb_stddev_entry.insert(0, "2")
    bb_stddev_entry.pack(pady=5)
    
    # Label and button for selecting color for High Band
    hb_color_label = ttk.Label(study_window, text="Selected High Band Color:")
    hb_color_label.pack(pady=5)
    hb_color_var = tk.StringVar(value='black')
    selected_hb_color_label = ttk.Label(study_window, background=hb_color_var.get(), width=2)
    selected_hb_color_label.pack(pady=5)
    hb_color_button = ttk.Button(study_window, text="Choose Color", command=lambda: select_color(hb_color_var,
                                 study_window, lambda: update_selected_bb_color('HBand')))
    hb_color_button.pack(pady=5)
    
    # Label and button for selecting color for Low Band
    lb_color_label = ttk.Label(study_window, text="Select Low Band Color:")
    lb_color_label.pack(pady=5)
    lb_color_var = tk.StringVar(value='black')
    selected_lb_color_label = ttk.Label(study_window, background=lb_color_var.get(), width=2)
    selected_lb_color_label.pack(pady=5)
    lb_color_button = ttk.Button(study_window, text="Choose Color", command=lambda: select_color(lb_color_var,
                                 study_window,lambda: update_selected_bb_color('LBand')))
    lb_color_button.pack(pady=5)
    
    # Label and button for selecting color for Middle Band
    mid_color_label = ttk.Label(study_window, text="Select Middle Band Color:")
    mid_color_label.pack(pady=5)
    mid_color_var = tk.StringVar(value='blue')
    selected_mid_color_label = ttk.Label(study_window, background=mid_color_var.get(), width=2)
    selected_mid_color_label.pack(pady=5)
    mid_color_button = ttk.Button(study_window, text="Choose Color", command=lambda: select_color(mid_color_var,
                                  study_window, lambda: update_selected_bb_color('BB_Mid')))
    mid_color_button.pack(pady=5)
    
    def update_selected_bb_color(line):
        if line == 'HBand':
            selected_color_label = hb_color_label
            selected_color_var = hb_color_var
        elif line == 'LBand':
            selected_color_label = lb_color_label
            selected_color_var = lb_color_var
        else:
            selected_color_label = mid_color_label
            selected_color_var = mid_color_var
            
        selected_color_label.config(background=selected_color_var.get())
        
    def on_ok():
        selected_bb_length = int(bb_length_entry.get())
        selected_bb_stddev = float(bb_stddev_entry.get())
        df_full['HBand'] = ta.volatility.bollinger_hband(df_full['Close'], window=selected_bb_length,
                                                         window_dev=selected_bb_stddev)
        update_chart_with_study('HBand', hb_color_var.get(), group='BB')
        df_full['LBand'] = ta.volatility.bollinger_lband(df_full['Close'], window=selected_bb_length,
                                                         window_dev=selected_bb_stddev)
        update_chart_with_study('LBand', lb_color_var.get(), group='BB')
        df_full['BB_Mid'] = ta.volatility.bollinger_mavg(df_full['Close'], window=selected_bb_length)
        update_chart_with_study('BB_Mid', mid_color_var.get(), group='BB')
        study_window.destroy()

    ok_button = ttk.Button(study_window, text="OK", command=on_ok)
    ok_button.pack(pady=10)  
    
    

function for adding stochastics

In [10]:
def add_stochastics_study(study_window):
    stoch_window_label = ttk.Label(study_window, text="Enter Stochastics Window Length:")
    stoch_window_label.pack(pady=5)

    stoch_window_entry = ttk.Entry(study_window, justify="center")
    stoch_window_entry.insert(0, "14")
    stoch_window_entry.pack(pady=5)

    stoch_smooth_label = ttk.Label(study_window, text="Enter Stochastics Smoothing Length:")
    stoch_smooth_label.pack(pady=5)

    stoch_smooth_entry = ttk.Entry(study_window, justify="center")
    stoch_smooth_entry.insert(0, "3")
    stoch_smooth_entry.pack(pady=5)
    
    # Label and button for selecting color for %K line
    stoch_k_color_label = ttk.Label(study_window, text="Select %K Line Color:")
    stoch_k_color_label.pack(pady=5)
    stoch_k_color_var = tk.StringVar(value='blue')
    selected_stoch_k_color_label = ttk.Label(study_window, background=stoch_k_color_var.get(), width=2)
    selected_stoch_k_color_label.pack(pady=5)
    stoch_k_color_button = ttk.Button(study_window, text="Choose Color", command=lambda: select_color(stoch_k_color_var,
                                      study_window, lambda: update_selected_stoch_color('%K')))
    stoch_k_color_button.pack(pady=5)

    # Label and button for selecting color for %D line
    stoch_d_color_label = ttk.Label(study_window, text="Select %D Line Color:")
    stoch_d_color_label.pack(pady=5)
    stoch_d_color_var = tk.StringVar(value='orange')
    selected_stoch_d_color_label = ttk.Label(study_window, background=stoch_d_color_var.get(), width=2)
    selected_stoch_d_color_label.pack(pady=5)
    stoch_d_color_button = ttk.Button(study_window, text="Choose Color", command=lambda: select_color(stoch_d_color_var,
                                      study_window, lambda: update_selected_stoch_color('%D')))
    stoch_d_color_button.pack(pady=5)
    
    def update_selected_stoch_color(component):
        if component == '%K':
            selected_color_label = selected_stoch_k_color_label
            selected_color_var = stoch_k_color_var
        else:
            selected_color_label = selected_stoch_d_color_label
            selected_color_var = stoch_d_color_var
            
        selected_color_label.config(background=selected_color_var.get())
        
    def on_ok():
        selected_stoch_window = int(stoch_window_entry.get())
        selected_stoch_smooth = int(stoch_smooth_entry.get())
        selected_stoch_k_color = stoch_k_color_var.get()
        selected_stoch_d_color = stoch_d_color_var.get()

        stoch = ta.momentum.StochasticOscillator(
            high=df_full['High'], 
            low=df_full['Low'], 
            close=df_full['Close'], 
            window=selected_stoch_window, 
            smooth_window=selected_stoch_smooth
        )

        df_full['%K'] = stoch.stoch_signal()
        update_chart_with_study('%K', selected_stoch_k_color, panel=1, group='Stochastics')
        df_full['%D'] = stoch.stoch()
        update_chart_with_study('%D', selected_stoch_d_color, panel=1, group='Stochastics')

        study_window.destroy()

    ok_button = ttk.Button(study_window, text="OK", command=on_ok)
    ok_button.pack(pady=10)
    
    

function for updating the chart with study

In [11]:
def update_chart_with_study(study_column, color, panel=0, type='line', width=0.5, group=None):
    
    global addplots
    
    chart_type = chart_type_var.get()
    if chart_type == 'renko' or chart_type == 'pnf':
        messagebox.showerror(
        "Add study error",
        "Studies cannot be added to renko or pnf charts. Please use candle, ohlc or line"
        )                 
        return
    
    if panel == 1:
        panel_populated = False
        for plot in addplots:
            if isinstance(plot, dict) and plot.get('panel') == panel:
                panel_populated = True
                break

        if panel_populated:
            addplots = [plot for plot in addplots if not (isinstance(plot, dict) and
                    plot.get('panel') == panel and plot.get('group') != group)]
            

    study_plot = mpf.make_addplot(df_full[study_column], type=type, color=color, width=width,panel=panel)
    study_plot['study'] = study_column
    study_plot['group'] = group
    addplots.append(study_plot)
    
    symbol = symbol_entry.get()
    fetch_and_display_chart(symbol, addplots=addplots)
    

function for clearing studies

In [12]:
def clear_all_studies():
    global addplots
    addplots = []
    symbol = symbol_entry.get()
    fetch_and_display_chart(symbol, addplots=addplots)

# Main Window

In [13]:
# Create the main application window
root = tk.Tk()
root.title("Stock Chart Viewer")

# Set the window to full screen
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
root.geometry(f"{screen_width}x{screen_height}+0+0")

# Make the window always on top
root.attributes('-topmost', True)
root.after_idle(root.attributes, '-topmost', False)

# Create a frame for the widgets on the left side
frame = ttk.Frame(root, width=int(screen_width * 0.2), height=screen_height)
frame.pack(side=tk.LEFT, fill=tk.Y)

# Create an entry widget for the stock symbol
symbol_label = ttk.Label(frame, text="Enter Stock Symbol:")
symbol_label.pack(pady=10)

symbol_entry = ttk.Entry(frame)
symbol_entry.pack(pady=10)

# Calculate the default start date (365 days back from today's date)
default_start_date = datetime.date.today() - datetime.timedelta(days=365)

# Create date entry widgets for the start and end dates
start_date_label = ttk.Label(frame, text="Select Start Date:")
start_date_label.pack(pady=10)

start_date_entry = DateEntry(frame, width=12, background='darkblue', foreground='white', borderwidth=2)
start_date_entry.set_date(default_start_date)
start_date_entry.pack(pady=10)

end_date_label = ttk.Label(frame, text="Select End Date:")
end_date_label.pack(pady=10)

end_date_entry = DateEntry(frame, width=12, background='darkblue', foreground='white', borderwidth=2)
end_date_entry.pack(pady=10)

# Create a drop-down menu for selecting the chart style
style_label = ttk.Label(frame, text="Select Chart Style:")
style_label.pack(pady=10)

style_options = ['binance', 'blueskies', 'brasil', 'charles', 'checkers', 'classic', 'default', 'ibd',
                 'kenan', 'mike', 'nightclouds', 'sas', 'starsandstripes', 'yahoo']

style_var = tk.StringVar(value='yahoo')  # Default style
style_menu = ttk.Combobox(frame, textvariable=style_var, values=style_options)
style_menu.pack(pady=10)

# Create a drop-down menu for selecting the chart type
chart_type_label = ttk.Label(frame, text="Select Chart Type:")
chart_type_label.pack(pady=10)

chart_type_options = ['ohlc', 'candle', 'line', 'renko', 'pnf']

chart_type_var = tk.StringVar(value='ohlc')  # Default chart type
chart_type_menu = ttk.Combobox(frame, textvariable=chart_type_var, values=chart_type_options)
chart_type_menu.pack(pady=10)

# Create a drop-down menu for selecting the studies
studies_label = ttk.Label(frame, text="Select Study:")
studies_label.pack(pady=10)

study_options = ['Moving Average', 'RSI', 'Bollinger Bands','MACD','Stochastics']

study_var = tk.StringVar(value='Moving Average')  # Default study
study_menu = ttk.Combobox(frame, textvariable=study_var, values=study_options)
study_menu.pack(pady=10)

# Create a button to add the selected study
add_study_button = ttk.Button(frame, text="Add Study", command = add_study)
add_study_button.pack(pady=10)

# Create a button to clear all studies
clear_studies_button = ttk.Button(frame, text="Clear All Studies",command=clear_all_studies) 
clear_studies_button.pack(pady=10)

# Create a button to fetch and display the chart
fetch_button = ttk.Button(frame, text="Fetch Chart", command = lambda : fetch_and_display_chart(symbol_entry.get()) )
fetch_button.pack(pady=10)

# Create an exit button
exit_button = ttk.Button(frame, text="Exit", command=root.destroy)
exit_button.pack(pady=10)

# Create a vertical separator
separator = ttk.Separator(root, orient='vertical')
separator.pack(side=tk.LEFT, fill=tk.Y, padx=10)

# Create a canvas to display the image in the main window (right side)
canvas = tk.Canvas(root, bg="white")
canvas.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)

# Create a scrollbar for the canvas
scrollbar = ttk.Scrollbar(root, orient=tk.VERTICAL, command=canvas.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

canvas.config(yscrollcommand=scrollbar.set)


# Start the tkinter main loop
root.mainloop()

[*********************100%%**********************]  1 of 1 completed
